In [2]:
import json
import pandas as pd
from datetime import datetime

In [3]:
# file_path = "../../data/raw/teslamotors_top_month_20250305.json"
def get_json(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        json_data = json.load(file_path)
    return json_data

In [4]:
def get_df(json):
    df = pd.DataFrame(json)
    return df

In [5]:
def manage_columns(df):
    df['created_utc'] = pd.to_datetime(df['created_utc'])
    df['year'] = df['created_utc'].dt.year
    df['month'] = df['created_utc'].dt.month
    df['day'] = df['created_utc'].dt.day
    df['time'] = df['created_utc'].dt.time
    df['day_of_week'] = df['created_utc'].dt.day_name()
    df = df.drop(columns=['url', 'id', 'permalink', 'author'])
    return df

In [6]:
def create_preprocessed_json(file_path_json):
    df = get_df(get_json(file_path_json))
    df = manage_columns(df)
    df['combined_text'] = df['title'].fillna('') + ' ' + df['body'].fillna('')# Combine text fields (title + body) for NLP models
    df['cleaned_text'] = df['combined_text'].str.replace(r'http\S+', '', regex=True).str.replace(r'\W+', ' ', regex=True)# Clean URLs and special chars from text


In [7]:
# Encode categorical data
df['flair_encoded'] = pd.factorize(df['flair'])[0]


NameError: name 'df' is not defined

In [ ]:
df.head()

,title,score,body,num_comments,flair,nsfw,awards,created_hour,created_dayofweek,combined_text,cleaned_text,flair_encoded
0,"Polestar's 'Trade In Your Tesla' $20,000 Deal ...",1211,,324,General,False,0,23,6,"Polestar's 'Trade In Your Tesla' $20,000 Deal ...",Polestar s Trade In Your Tesla 20 000 Deal Is ...,0
1,Can confirm it looks better in real life,906,,182,Vehicles - Model Y,False,0,4,6,Can confirm it looks better in real life,Can confirm it looks better in real life,1
2,New Model Y Ultra Red,904,New Model Y Ultra Red photos from Tesla Meatpa...,149,Vehicles - Model Y,False,0,22,0,New Model Y Ultra Red New Model Y Ultra Red ph...,New Model Y Ultra Red New Model Y Ultra Red ph...,1
3,Toronto Drops Tesla from Taxi Grant Program as...,789,,183,General,False,0,0,3,Toronto Drops Tesla from Taxi Grant Program as...,Toronto Drops Tesla from Taxi Grant Program as...,0
4,Tesla and Steak ‘n Shake partner to roll out 1...,783,,86,Energy - Charging,False,0,21,4,Tesla and Steak ‘n Shake partner to roll out 1...,Tesla and Steak n Shake partner to roll out 10...,2


In [ ]:
import os

# Ensure the directory exists
save_dir = r"../../data/processed"

# Save to CSV
csv_path = os.path.join(save_dir, 'preprocessed_data.csv')
json_path = os.path.join(save_dir, 'preprocessed_data.json')
df.to_csv(csv_path, index=False)
# df.to_json(csv_path.replace('.csv', '.json'), orient='records', lines=True)
df.to_json(json_path, orient='records', indent=4) 
